# Testing spaCy

Experimenting with spaCy, a python package for NLP: https://spacy.io/

In [ ]:
import sys
from tqdm import tqdm_notebook
import tqdm
import spacy
from spacy import displacy

In [ ]:
sys.executable

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp_en = spacy.load('en')

# Load Portuguese tokenizer, tagger, parser, NER and word vectors
nlp_pt = spacy.load('pt')

In [ ]:
# Colors for the displacy entity visualizer
colors = {'ORG': '#bc8825',
          'PER': '#5df322',
          'PERSON': '#5df322',
          'DATE': '#3399ff',
          'Norp': '#ec134b',
          'MISC': '#6f6f6f',
          'LOC': '#ff81b0'
          }

# Color and entity options for the displacy visualizer
options = {'ents': ['ORG', 'PER', 'PERSON', 'DATE', 'Norp', 'MISC', 'LOC'], 'colors': colors}

### Example english short text

In [ ]:
text = (u"When Sebastian Thrun started working on self-driving cars at "
        u"Google in 2007, few people outside of the company took him "
        u"seriously. “I can tell you very senior CEOs of major American "
        u"car companies would shake my hand and turn away because I wasn’t "
        u"worth talking to,” said Thrun, now the co-founder and CEO of "
        u"online higher education startup Udacity, in an interview with "
        u"Recode earlier this week.")

In [ ]:
doc = nlp_en(text)

In [ ]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
# Only get the people entities
for entity in doc.ents:
    if 'PER' in entity.label_:
        print(entity.text)

In [ ]:
# Visualize the entities of the text in the (beautiful :) ) displacy visualizer
displacy.render(doc, style='ent', jupyter=True, options=options)

In [ ]:
# Determine semantic similarities
doc1 = nlp_en(u"my fries were super gross")
doc2 = nlp_en(u"such disgusting fries")
similarity = doc1.similarity(doc2)
print(doc1.text, doc2.text, similarity)

### Example portuguese short text

In [ ]:
text = (u"Quando as revelações atingem realidades que nos são próximas, é mais difícil "
        u"analisá-las com o distanciamento e a frieza com que analisamos outras. O caso "
        u"do Football Leaks – que, ainda por cima, mexe com o tão nacionalmente adorado "
        u"futebol – é um paradigma disso mesmo, com a animosidade a sobrepor-se à racionalidade, "
        u"e os sentimentos de invasão e traição a sobreporem-se ao eventual valor social, "
        u"político e institucional das revelações. Se é Rui Pinto o homem responsável pelo "
        u"Football Leaks, não será ele uma espécie de Julian Assange do futebol mundial? E digo "
        u"Assange para usar um nome concreto que poderia ser qualquer outro. No fundo, não serão "
        u"os Football Leaks tão saudáveis para a justiça no desporto como foram Wikileaks, Panama "
        u"Papers, Paradise Papers ou outras libertações de documentos do género? O que distingue "
        u"este whistleblower de outros?")

In [ ]:
doc = nlp_pt(text)

In [ ]:
# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
# Only get the people entities
for entity in doc.ents:
    if 'PER' in entity.label_:
        print(entity.text)

In [ ]:
# Visualize the entities of the text in the (beautiful :) ) displacy visualizer
displacy.render(doc, style='ent', jupyter=True, options=options)

### Training a model

In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import spacy
from spacy.util import minibatch, compounding

In [ ]:
def load_data(limit=0, split=0.8):
    """Load data from the IMDB dataset."""
    # Partition off part of the train data for evaluation
    train_data, _ = thinc.extra.datasets.imdb()
    random.shuffle(train_data)
    train_data = train_data[-limit:]
    texts, labels = zip(*train_data)
    cats = [{'POSITIVE': bool(y)} for y in labels]
    split = int(len(train_data) * split)
    return (texts[:split], cats[:split]), (texts[split:], cats[split:])

In [ ]:
def evaluate(tokenizer, textcat, texts, cats):
    docs = (tokenizer(text) for text in texts)
    tp = 1e-8  # True positives
    fp = 1e-8  # False positives
    fn = 1e-8  # False negatives
    tn = 1e-8  # True negatives
    for i, doc in enumerate(textcat.pipe(docs)):
        gold = cats[i]
        for label, score in doc.cats.items():
            if label not in gold:
                continue
            if score >= 0.5 and gold[label] >= 0.5:
                tp += 1.
            elif score >= 0.5 and gold[label] < 0.5:
                fp += 1.
            elif score < 0.5 and gold[label] < 0.5:
                tn += 1
            elif score < 0.5 and gold[label] >= 0.5:
                fn += 1
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f_score = 2 * (precision * recall) / (precision + recall)
    return {'textcat_p': precision, 'textcat_r': recall, 'textcat_f': f_score}

In [ ]:
#!/usr/bin/env python
# coding: utf8
"""Train a convolutional neural network text classifier on the
IMDB dataset, using the TextCategorizer component. The dataset will be loaded
automatically via Thinc's built-in dataset loader. The model is added to
spacy.pipeline, and predictions are available via `doc.cats`. For more details,
see the documentation:
* Training: https://spacy.io/usage/training

Compatible with: spaCy v2.0.0+
"""
def main(model=None, output_dir=None, n_iter=20, n_texts=2000):
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # add the text classifier to the pipeline if it doesn't exist
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'textcat' not in nlp.pipe_names:
        textcat = nlp.create_pipe('textcat')
        nlp.add_pipe(textcat, last=True)
    # otherwise, get it, so we can add labels to it
    else:
        textcat = nlp.get_pipe('textcat')

    # add label to text classifier
    textcat.add_label('POSITIVE')

    # load the IMDB dataset
    print("Loading IMDB data...")
    (train_texts, train_cats), (dev_texts, dev_cats) = load_data(limit=n_texts)
    print("Using {} examples ({} training, {} evaluation)"
          .format(n_texts, len(train_texts), len(dev_texts)))
    train_data = list(zip(train_texts,
                          [{'cats': cats} for cats in train_cats]))

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'textcat']
    with nlp.disable_pipes(*other_pipes):  # only train textcat
        optimizer = nlp.begin_training()
        print("Training the model...")
        print('{:^5}\t{:^5}\t{:^5}\t{:^5}'.format('LOSS', 'P', 'R', 'F'))
        for i in range(n_iter):
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(train_data, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.2,
                           losses=losses)
            with textcat.model.use_params(optimizer.averages):
                # evaluate on the dev data split off in load_data()
                scores = evaluate(nlp.tokenizer, textcat, dev_texts, dev_cats)
            print('{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}'  # print a simple table
                  .format(losses['textcat'], scores['textcat_p'],
                          scores['textcat_r'], scores['textcat_f']))

    # test the trained model
    test_text = "This movie sucked"
    doc = nlp(test_text)
    print(test_text, doc.cats)

    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        print(test_text, doc2.cats)

In [ ]:
# Run the training
main(model=None, output_dir=None, n_iter=20, n_texts=2000)